In [99]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import datetime
import os
import sys
import dateutil.parser

from __future__ import print_function

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [100]:
sys.path.insert(0, os.path.abspath('..'))

from UKMovementSensing import dataprep

## Filepaths

In [101]:
rootpath = '/media/sf_VBox_Shared/London/raw/first5/'
annotations_path = rootpath + 'tud_first5.csv'
wearcodes_path = rootpath + 'wearcodes.csv'

In [102]:
filepath = rootpath + 'accelerometer_5second/'

## Functions

## Load and preprocess annotations

In [103]:
annotations = dataprep.process_annotations(annotations_path)
print(annotations.shape)
annotations.head()

Number of missing start and end times: 0 0
Differens between start and end not always 10 minutes! in 10 cases.
(1440, 5)


,serflag,start_time,end_time,act,act_label
0,52920071_100,2015-03-01 04:00:00+00:00,2015-03-01 04:10:00+00:00,1.0,Sleeping and resting (including sick in bed)
1,52920071_100,2015-03-01 04:10:00+00:00,2015-03-01 04:20:00+00:00,1.0,Sleeping and resting (including sick in bed)
2,52920071_100,2015-03-01 04:20:00+00:00,2015-03-01 04:30:00+00:00,1.0,Sleeping and resting (including sick in bed)
3,52920071_100,2015-03-01 04:30:00+00:00,2015-03-01 04:40:00+00:00,1.0,Sleeping and resting (including sick in bed)
4,52920071_100,2015-03-01 04:40:00+00:00,2015-03-01 04:50:00+00:00,1.0,Sleeping and resting (including sick in bed)


## Join with wearcodes

In [104]:
wearcodes = pd.read_csv(wearcodes_path)
wearcodes['Day1'] = [datetime.datetime.strptime(s, '%d/%m/%Y') for s in wearcodes['Day1']]
wearcodes['Day2'] = [datetime.datetime.strptime(s, '%d/%m/%Y') for s in wearcodes['Day2']]
print(wearcodes.shape)
wearcodes.head()

(5, 8)


,Monitor,Day1,Day2,binFile,serflag,fileSize,day1Ready,day2Ready
0,16566,2015-03-11,2015-03-15,__016566_2015-03-24 14-52-30.bin,81510041_100,777664025,True,True
1,16580,2015-03-05,2015-03-08,__016580_2015-03-12 09-09-15.bin,102650051_100,522410444,True,True
2,18618,2015-03-15,2015-03-16,__018618_2015-03-20 11-15-03.bin,62980071_100,565693441,True,True
3,18635,2015-03-01,2015-03-03,__018635_2015-03-11 14-06-25.bin,52920071_100,692552249,True,True
4,18643,2015-03-06,2015-03-07,__018643_2015-03-13 12-19-47.bin,122650071_100,525756098,True,True


In [105]:
annotations_codes = pd.merge(annotations, wearcodes, on='serflag', how='left')
print(annotations_codes.shape)
annotations_codes.head()

(1440, 12)


,serflag,start_time,end_time,act,act_label,Monitor,Day1,Day2,binFile,fileSize,day1Ready,day2Ready
0,52920071_100,2015-03-01 04:00:00+00:00,2015-03-01 04:10:00+00:00,1.0,Sleeping and resting (including sick in bed),18635,2015-03-01,2015-03-03,__018635_2015-03-11 14-06-25.bin,692552249,True,True
1,52920071_100,2015-03-01 04:10:00+00:00,2015-03-01 04:20:00+00:00,1.0,Sleeping and resting (including sick in bed),18635,2015-03-01,2015-03-03,__018635_2015-03-11 14-06-25.bin,692552249,True,True
2,52920071_100,2015-03-01 04:20:00+00:00,2015-03-01 04:30:00+00:00,1.0,Sleeping and resting (including sick in bed),18635,2015-03-01,2015-03-03,__018635_2015-03-11 14-06-25.bin,692552249,True,True
3,52920071_100,2015-03-01 04:30:00+00:00,2015-03-01 04:40:00+00:00,1.0,Sleeping and resting (including sick in bed),18635,2015-03-01,2015-03-03,__018635_2015-03-11 14-06-25.bin,692552249,True,True
4,52920071_100,2015-03-01 04:40:00+00:00,2015-03-01 04:50:00+00:00,1.0,Sleeping and resting (including sick in bed),18635,2015-03-01,2015-03-03,__018635_2015-03-11 14-06-25.bin,692552249,True,True


## Load and preprocess data

In [106]:
byName = annotations_codes.groupby('binFile')
dfs = {}
for name, fileAnnotations in byName:
    # Load data
    df = dataprep.loadData(name, filepath)
    
    if df is not None:    
        # Add annotations
        dfs.update(df)

In [107]:
test_name = dfs.keys()[0]
dfs[test_name].head()

,acceleration,anglex,angley,anglez,invalid,filename
timestamp,,,,,,
2015-03-15 04:00:00,0.001881,23.5825747869742,-31.9813789387999,48.4125825249496,0,__018618_2015-03-20 11-15-03.bin_day1
2015-03-15 04:00:05,0.001864,23.6166016121363,-31.8864189902329,48.4733538278626,0,__018618_2015-03-20 11-15-03.bin_day1
2015-03-15 04:00:10,0.001593,23.6326325559709,-31.8533596832388,48.4913226097319,0,__018618_2015-03-20 11-15-03.bin_day1
2015-03-15 04:00:15,0.001915,23.5976632507158,-31.8753685358903,48.4973484430271,0,__018618_2015-03-20 11-15-03.bin_day1
2015-03-15 04:00:20,0.001773,23.7701111010191,-31.9568528087253,48.2958632383147,0,__018618_2015-03-20 11-15-03.bin_day1


In [108]:
print(wearcode.to_dict()['Day1'].values()[0].date())

2015-02-22


In [111]:
#wearcodes.groupby('binFile')
for binFile, day in dfs.keys():
    df = dfs[(binFile, day)]
    wearcode = wearcodes[wearcodes['binFile']==binFile]
    realday = wearcode.to_dict()['Day'+str(day)].values()[0].date()
    file_date = df.index[0].date() 
    if file_date !=  realday:
        print(binFile+'_day'+str(day)+"\t"+str(file_date)+"\t"+str(realday))

__018635_2015-03-11 14-06-25.bin_day2	2015-03-01	2015-03-03
__016566_2015-03-24 14-52-30.bin_day2	2015-03-11	2015-03-15
__018643_2015-03-13 12-19-47.bin_day2	2015-03-06	2015-03-07
__018618_2015-03-20 11-15-03.bin_day2	2015-03-15	2015-03-16
__016580_2015-03-12 09-09-15.bin_day2	2015-03-05	2015-03-08


In [ ]:
for binFile, day in dfs.keys():
    df = dfs[(binFile, day)]
    annotations_group = byName.get_group(binFile)
    #annotations_group_day = annotations_group[annotations_group['Day'+day]]
    for idx in annotations_group.index:
        rStart = annotations_group.loc[idx, 'start_time']
        rEnd = annotations_group.loc[idx, 'end_time']
        df.loc[(df.index < rEnd) & (df.index > rStart), 'Activity'] = annotations_group.loc[idx, 'act_label']

In [ ]:
df

In [ ]:
merged_directory = os.path.join(rootpath, 'merged_5sec')
if not os.path.exists(merged_directory):
    os.makedirs(merged_directory)

In [ ]:
for binfile, day in dfs.keys():
    df = dfs[(binFile, day)]
    df.to_csv(os.path.join(merged_directory, binfile+'_day'+day+'.csv'))